In [ ]:
from math import sqrt
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import  RandomForestRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials, space_eval

# Read data and Separate features and target variable
data = pd.read_excel('../data/data-3.xlsx')
X = data.drop(['PFAS', 'LogKd'], axis=1)
y = data['LogKd']
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

final_models = {}

# Define models and hyperparameter spaces
model_spaces = {'RF': {'model': RandomForestRegressor,'params': {
            'n_estimators': hp.choice('rf_n_estimators', range(100, 1001)),
            'max_depth': hp.choice('rf_max_depth', range(1, 21)),
            'min_samples_split': hp.choice('rf_min_samples_split', range(2, 11)),
            'min_samples_leaf': hp.choice('rf_min_samples_leaf', range(1, 11)),
            'max_features': hp.choice('rf_max_features', ['sqrt', 'log2', None]),
            'bootstrap': hp.choice('rf_bootstrap', [True, False])}}}

# Define objective function
def objective(params, model, X, y):
    reg = model(**params)
    kf = KFold(n_splits=5)
    scores = cross_val_score(reg, X, y, cv=kf, scoring='neg_mean_squared_error')
    return {'loss': -np.mean(scores), 'status': STATUS_OK}

# Optimize hyperparameters
for model_name, space in model_spaces.items():
    print(f"Optimizing {model_name}")
    trials = Trials()
    best = fmin(fn=lambda params: objective(params, space['model'], X_train_full, y_train_full),
                space=space['params'], algo=tpe.suggest, max_evals=50, trials=trials,rstate=np.random.default_rng(42))
    
    best_params = space_eval(space['params'], best)
    print(f"Best params for {model_name}: {best_params}")
    
    # Evaluate the model with the best hyperparameters
    model = space['model'](**best_params)
    
    model.fit(X_train_full, y_train_full)
    y_pred = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_pred)) 
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"{model_name} on test set - RMSE: {rmse}, MAE: {mae}, R^2: {r2}\n")

    # Save the best model
    model = space['model'](**best_params)
    model.fit(X_train_full, y_train_full)
    final_models[model_name] = model

# Save the model in a .pkl file
rf_model = final_models['RF']  
joblib.dump(rf_model, 'rf_model.pkl')

In [2]:
# Predict using the saved model
loaded_rf_model = joblib.load('rf_model.pkl')

data_application = pd.read_excel('../data/data-4.xlsx')

# Repeat prediction 100 times and take the average
predictions = []
for _ in range(100):
    y_pred = loaded_rf_model.predict(data_application)
    predictions.append(y_pred)

# Caculate mean and save
mean_predictions = np.mean(predictions, axis=0)
predictions_df = pd.DataFrame({'LogKd_Predicted': mean_predictions})
predictions_df.to_excel('../data/data-4_Predictions.xlsx', index=False)

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys, DataStructs
df = pd.read_excel('../data/SMILES.xlsx')
                   
# definition the Tanimoto similarity function
def calculate_tanimoto(smiles1, smiles2):
    try:
        mol1 = Chem.MolFromSmiles(str(smiles1))
        mol2 = Chem.MolFromSmiles(str(smiles2))
        if mol1 is None or mol2 is None:
            return 0.0
        fp1 = MACCSkeys.GenMACCSKeys(mol1)
        fp2 = MACCSkeys.GenMACCSKeys(mol2)
        return DataStructs.FingerprintSimilarity(fp1, fp2)
    except:
        return 0.0

# Calculate the maximum Tanimoto similarity between the new molecule and all the training set molecules
max_tanimoto_indices = []
for new_smiles in df.iloc[:, 1]:
    max_similarity = 0
    for train_smiles in df.iloc[:, 0]:
        similarity = calculate_tanimoto(train_smiles, new_smiles)
        if similarity > max_similarity:
            max_similarity = similarity
    max_tanimoto_indices.append(max_similarity)

# Save to DataFrame
df['Max Tanimoto Index'] = max_tanimoto_indices
   
# Save to Excel
output_file = '../data/SMILES_with_Max_Tanimoto.xlsx'
df.to_excel(output_file, index=False)
print(f"Completed {output_file}")

In [ ]:
import joblib
import pandas as pd

# Predict using the saved RF model
loaded_rf_model = joblib.load('rf_model.pkl')
data_application = pd.read_excel('data.xlsx')

# Repeat prediction 100 times and take the averag
predictions = []
for _ in range(100):
    y_pred = loaded_rf_model.predict(data_application)
    predictions.append(y_pred)

# Caculate mean and save
mean_predictions = np.mean(predictions, axis=0)
predictions_df = pd.DataFrame({'LogKd_Predicted': mean_predictions})
predictions_df.to_excel('data_Predictions.xlsx', index=False)